In [ ]:
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
from tensorflow import keras
import tensorflow as tf
import numpy
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import roc_curve, precision_recall_curve, auc
import sklearn.metrics as metrics
from tensorflow.keras.models import load_model
import os
import sklearn
import warnings
warnings.filterwarnings('ignore',category=UserWarning)

x_path = '../Generated_data/test_set.npz'
y_path = '../Generated_data/one_hot_test.csv'
model_path = 'Model_save/model_final.h5'

def get_accuracy(y_test, pred_score, label):
    pred_z = []
    
    for i in tqdm_notebook(range(len(pred_score))):
        per_max = max(pred_score[i])
        pred_z.append(int(label[np.where(pred_score[i]==per_max)[0][0]]))
        
    return accuracy_score(y_test, pred_z)
    
def get_roc(one_hot_test, pred_score, class_label):
    return roc_auc_score(one_hot_test,pred_score, multi_class='raise', average='weighted')

def get_pr(one_hot_test, pred_score, label):
    
    precision = dict()
    recall = dict()
    pr_score = dict()
    
    for i in range(len(label)):
        precision[i], recall[i], _ = precision_recall_curve(one_hot_test.iloc[:, i], pred_score[:, i])
        pr_score[i] = auc(recall[i], precision[i])
        
    return np.mean(list(pr_score.values()))

def evaluation(y_test, one_hot_test, pred_score, label):
    accuracy = get_accuracy(y_test, pred_score, label)
    roc = get_roc(one_hot_test, pred_score, label)
    pr = get_pr(one_hot_test, pred_score, label)
    
    print(f'Accuracy:{accuracy}')
    print(f'AUROC:{roc}')
    print(f'AUPR:{pr}')
    
def get_prediction(x_path, y_path, model_path):
     
    test_set = np.load(x_path)

    test_A = test_set['test_A']
    test_B = test_set['test_B']

    one_hot_test = pd.read_csv(y_path, index_col=0)

    temp = one_hot_test.reset_index(drop=True)

    label = list(one_hot_test.columns)

    y_test = []

    for i in tqdm_notebook(range(len(temp))):
        y_test.append(int(label[list(temp.iloc[i]).index(1)]))

    model_best = load_model(model_path, compile=False)
    model_best.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc')])

    pred_z = model_best.predict({'inputA':test_A, 'inputB':test_B})
    
    return y_test, one_hot_test, pred_z, label


y_test, one_hot_test, pred_z, label = get_prediction(x_path, y_path, model_path)

evaluation(y_test, one_hot_test, pred_z, label)